# Imports


In [1]:
import os
import torch
import torch.nn as nn
import wandb
import numpy as np

from pipelines.transform import transform
from model.scheduler import GradualWarmupScheduler
from model.cls_head import ClassifierHead
from pipelines.dataset import SignDataset
from torch.utils.data import DataLoader
from torchvision.models import vit_b_32

In [5]:
def top_k_accuracy(scores, labels, topk=(1, )):
    """Calculate top k accuracy score.
    Args:
        scores (list[np.ndarray]): Prediction scores for each class.
        labels (list[int]): Ground truth labels.
        topk (tuple[int]): K value for top_k_accuracy. Default: (1, ).
    Returns:
        list[float]: Top k accuracy score for each k.
    """
    res = np.zeros(len(topk))
    labels = np.array(labels)[:, np.newaxis]
    for i, k in enumerate(topk):
        max_k_preds = np.argsort(scores, axis=1)[:, -k:][:, ::-1]
        match_array = np.logical_or.reduce(max_k_preds == labels, axis=1)
        topk_acc_score = match_array.sum() / match_array.shape[0]
        res[i] = topk_acc_score
    return res


def train_one_epoch(epoch_index, interval=5):
    """Run one epoch for training.
    Args:
        epoch_index (int): Current epoch.
        interval (int): Frequency at which to print logs.
    Returns:
        last_loss (float): Loss value for the last batch.
    """
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, (rgb, targets) in enumerate(train_loader):
        rgb, targets = rgb.to(device), targets.to(device)
        
        targets = targets.reshape(-1, )

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(rgb)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, targets)
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), max_norm=40, norm_type=2.0)

        # Adjust learning weights
        optimizer.step()
        

        # Gather data and report
        running_loss += loss.item()
        if i % interval == interval-1:
            last_loss = running_loss / interval  # loss per batch
            print(
                f'Epoch [{epoch_index}][{i+1}/{len(train_loader)}], lr: {scheduler.get_last_lr()[0]:.5e}, loss: {last_loss:.5}')
            running_loss = 0.

    return last_loss, scheduler.get_last_lr()[0]


def validate():
    """Run one epoch for validation.
    Returns:
        avg_vloss (float): Validation loss value for the last batch.
        top1_acc (float): Top-1 accuracy in decimal.
        top5_acc (float): Top-5 accuracy in decimal.
    """
    running_vloss = 0.0
    running_vacc = np.zeros(2)

    print('Evaluating top_k_accuracy...')

    with torch.inference_mode():
        for i, (rgb, targets)  in enumerate(test_loader):
            rgb, targets = rgb.to(device), targets.to(device)
            
            targets = targets.reshape(-1, )

            outputs = model(rgb)

            loss = loss_fn(outputs, targets)
            running_vloss += loss

            running_vacc += top_k_accuracy(outputs.detach().cpu().numpy(),
                                           targets.detach().cpu().numpy(), topk=(1, 5))

    avg_vloss = running_vloss / (i + 1)

    acc = running_vacc/len(test_loader)
    top1_acc = acc[0].item()
    top5_acc = acc[1].item()

    return (avg_vloss, top1_acc, top5_acc)

if __name__ == "__main__":
#     wandb.init(entity="cares",
#               project="seven-sees",
#               group="v1")

    device='cuda'

    work_dir = 'work_dirs/wlasl/vit/'
    batch_size = 1
    startepoch = 0
    os.makedirs(work_dir, exist_ok=True)

    transforms_train = transform(mode = 'train')
    transforms_test = transform(mode = 'test')

    train_dataset = SignDataset(ann_file='../data/wlasl/wlasl_amaglgam_train.JSON',
                     root_dir='../data/wlasl/rawframes/',
                     split='train',
                     clip_len=16,
                     frame_interval=4,
                     num_clips=1,
                     resolution=224,
                     test_mode=False,
                        transforms = transforms_train)

    test_dataset = SignDataset(ann_file='../data/wlasl/wlasl_amaglgam_val.JSON',
                     root_dir='../data/wlasl/rawframes/',
                     split='val',
                     clip_len=16,
                     frame_interval=4,
                     num_clips=1,
                     resolution=224,
                     test_mode=True,
                        transforms = transforms_test)

    # Setting up dataloaders
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=12,
                                        pin_memory=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=1,
                                        shuffle=True,
                                        num_workers=12,
                                        pin_memory=True)


    model = nn.Sequential(vit_b_32(weights='DEFAULT'),
                         ClassifierHead(16, 4000))

    if(startepoch>0):
        print("resuming from epoch %i"%startepoch)
        model.load_state_dict(torch.load("work_dirs/wlasl/seven-seas-v1/epoch_" + str(startepoch) + ".pth"))
    

    # Specify optimizer
    optimizer = torch.optim.SGD(
        model.parameters(), lr=0.0000125, momentum=0.9, weight_decay=0.000001)

    # Specify Loss
    loss_cls = nn.CrossEntropyLoss()

    # Specify total epochs
    epochs = 500

    # Specify learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=120, gamma=0.1)

    scheduler_steplr = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[34, 84], gamma=0.1)
    # scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=16, after_scheduler=scheduler_steplr)
    scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=1, after_scheduler=scheduler_steplr)


    # Specify Loss
    loss_fn = nn.CrossEntropyLoss()

    # Setup wandb
#     wandb.watch(model, log_freq=10)


    # Transfer model to device
    model.to(device)

    for epoch in range(startepoch,epochs,1):
        # Turn on gradient tracking and do a forward pass
        model.train(True)
        avg_loss, learning_rate = train_one_epoch(epoch+1)

        # Turn off  gradients for reporting
        model.train(False)

        avg_vloss, top1_acc, top5_acc = validate()

        print(
            f'top1_acc: {top1_acc:.4}, top5_acc: {top5_acc:.4}, train_loss: {avg_loss:.5}, val_loss: {avg_vloss:.5}')

        # Track best performance, and save the model's state
        model_path = work_dir + f'epoch_{epoch+1}.pth'
        print(f'Saving checkpoint at {epoch+1} epochs...')
        torch.save(model.state_dict(), model_path)

         # Adjust learning rate
        scheduler.step()
 
        # Track wandb
#         wandb.log({'train/loss': avg_loss,
#                    'train/learning_rate': learning_rate,
#                    'val/loss': avg_vloss,
#                    'val/top1_accuracy': top1_acc,
#                    'val/top5_accuracy': top5_acc})

ValueError: too many values to unpack (expected 4)

In [4]:
n c t h w

NameError: name 'rgb' is not defined